In [1]:
import numpy as np
import pandas as pd

from geopy.geocoders import Nominatim

## Wildfire Data

In [2]:
geolocator = Nominatim(user_agent="example app")
Counties = ['Alameda','Alpine','Amador','Butte','Calaveras','Colusa','Contra Costa','Del Norte','El Dorado','Fresno','Glenn','Humboldt','Imperial','Inyo','Kern','Kings','Lake','Lassen','Los Angeles','Madera','Marin','Mariposa','Mendocino','Merced','Modoc','Mono','Monterey','Napa','Nevada','Orange','Placer','Plumas','Riverside','Sacramento','San Benito','San Bernardino','San Diego','San Francisco','San Joaquin','San Luis Obispo','San Mateo','Santa Barbara','Santa Clara','Santa Cruz','Shasta','Sierra','Siskiyou','Solano','Sonoma','Stanislaus','Sutter','Tehama','Trinity','Tulare','Tuolumne','Ventura','Yolo','Yuba']

def retCounty(a,x,y):
    try:
        adrs = geolocator.reverse(str(x)+','+str(y))
        if(adrs==None):
            if isinstance(a, str):
                return a
            else:
                print(str(x)+','+str(y), adrs, a)
                return ''

        adrs = adrs[0].split(',')
        for x in Counties:
            for w in adrs:
                if x in w:
                    return x
                
        if isinstance(a, str):
            return a
        else:
            print(str(x)+','+str(y), a)
            return ''


    except:
        if isinstance(a, str):
            return a
        else:
            print(str(x)+','+str(y), a)
            return ''
        

retCounty('?',40.774000,-122.309000)

'Shasta'

In [ ]:
data = pd.read_csv('./mapdataall.csv')
my_data = data[['incident_latitude','incident_longitude','incident_dateonly_created','incident_control','incident_containment','incident_acres_burned','incident_county','incident_name','calfire_incident','is_active','incident_id','incident_type']]
my_data['County'] = my_data.apply(lambda row: retCounty(row.incident_county,row.incident_latitude,row.incident_longitude), axis=1)

my_data.to_csv('Processed_County.csv')

In [4]:
data = pd.read_csv('./mapdataall.csv')
data.columns

Index(['incident_name', 'incident_is_final', 'incident_date_last_update',
       'incident_date_created', 'incident_administrative_unit',
       'incident_administrative_unit_url', 'incident_county',
       'incident_location', 'incident_acres_burned', 'incident_containment',
       'incident_control', 'incident_cooperating_agencies',
       'incident_longitude', 'incident_latitude', 'incident_type',
       'incident_id', 'incident_url', 'incident_date_extinguished',
       'incident_dateonly_extinguished', 'incident_dateonly_created',
       'is_active', 'calfire_incident', 'notification_desired'],
      dtype='object')

In [5]:
my_data = pd.read_csv('./Processed_County.csv')
my_data["data_id"] = my_data["County"] + '-'+ pd.to_datetime(my_data['incident_dateonly_created'].copy()).dt.strftime('%Y-%m').astype(str)
my_data = my_data.drop(columns=['incident_dateonly_created','incident_county','incident_latitude','incident_longitude'])
my_data.reset_index()
wildfire_data = my_data.groupby(by='data_id').sum()
wildfire_data = wildfire_data.dropna()
wildfire_data = wildfire_data[['incident_containment','incident_acres_burned','calfire_incident']]

In [6]:
wildfire_data

,incident_containment,incident_acres_burned,calfire_incident
data_id,,,
Alameda-2013-07,200.0,88.0,2
Alameda-2016-06,100.0,211.0,1
Alameda-2017-08,200.0,244.0,1
Alameda-2017-09,200.0,177.0,1
Alameda-2017-10,100.0,116.0,1
...,...,...,...
Yuba-2020-03,0.0,0.0,0
Yuba-2020-07,100.0,500.0,0
Yuba-2020-08,100.0,600.0,0


In [7]:
wildfire_data.to_csv('Weather_data.csv')

## Weather Dataset

In [8]:
import requests
import json
import time
import csv

## Data Scraping
- <mark>Note: replace api-key by the one you acquired<mark>

In [20]:
Counties = ['Alameda','Alpine','Amador','Butte','Calaveras','Colusa','Contra Costa','Del Norte','El Dorado','Fresno','Glenn','Humboldt','Imperial','Inyo','Kern','Kings','Lake','Lassen','Los Angeles','Madera','Marin','Mariposa','Mendocino','Merced','Modoc','Mono','Monterey','Napa','Nevada','Orange','Placer','Plumas','Riverside','Sacramento','San Benito','San Bernardino','San Diego','San Francisco','San Joaquin','San Luis Obispo','San Mateo','Santa Barbara','Santa Clara','Santa Cruz','Shasta','Sierra','Siskiyou','Solano','Sonoma','Stanislaus','Sutter','Tehama','Trinity','Tulare','Tuolumne','Ventura','Yolo','Yuba']
for i,loc in enumerate(Counties):
    r = requests.get('https://api.oikolab.com/weather',
                     params={'param': ['temperature','wind_speed','total_precipitation','snowfall','relative_humidity','surface_pressure','humidex'],
                             'start': '2013-01-01',
                             'end': '2020-12-31',
                             'freq': 'D',
                             'location': loc,
                             'api-key': 'e0bd5274d90f45ad9bb43fd6b9037238'}
                     )
    weather_data = json.loads(r.json()['data'])
    df = pd.DataFrame(index=pd.to_datetime(weather_data['index'], 
                                           unit='s'),
                      data=weather_data['data'],
                      columns=weather_data['columns'])
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'date'})
    df.to_csv(loc+'.csv',index=True)
    print(f'{i}: {loc}')
    if(i%10==9):
        time.sleep(300)

0: Alameda
1: Alpine
2: Amador
3: Butte
4: Calaveras
5: Colusa
6: Contra Costa
7: Del Norte
8: El Dorado
9: Fresno
10: Glenn
11: Humboldt
12: Imperial
13: Inyo
14: Kern
15: Kings
16: Lake
17: Lassen
18: Los Angeles
19: Madera
20: Marin
21: Mariposa
22: Mendocino
23: Merced
24: Modoc
25: Mono
26: Monterey
27: Napa
28: Nevada
29: Orange
30: Placer
31: Plumas
32: Riverside
33: Sacramento
34: San Benito
35: San Bernardino
36: San Diego
37: San Francisco
38: San Joaquin
39: San Luis Obispo
40: San Mateo
41: Santa Barbara
42: Santa Clara
43: Santa Cruz
44: Shasta
45: Sierra
46: Siskiyou
47: Solano
48: Sonoma
49: Stanislaus
50: Sutter
51: Tehama
52: Trinity
53: Tulare
54: Tuolumne
55: Ventura
56: Yolo
57: Yuba


#### Data Cleaning

In [23]:
Counties = ['Alameda','Alpine','Amador','Butte','Calaveras','Colusa','Contra Costa','Del Norte','El Dorado','Fresno','Glenn','Humboldt','Imperial','Inyo','Kern','Kings','Lake','Lassen','Los Angeles','Madera','Marin','Mariposa','Mendocino','Merced','Modoc','Mono','Monterey','Napa','Nevada','Orange','Placer','Plumas','Riverside','Sacramento','San Benito','San Bernardino','San Diego','San Francisco','San Joaquin','San Luis Obispo','San Mateo','Santa Barbara','Santa Clara','Santa Cruz','Shasta','Sierra','Siskiyou','Solano','Sonoma','Stanislaus','Sutter','Tehama','Trinity','Tulare','Tuolumne','Ventura','Yolo','Yuba']

for loc in Counties:
    tmp_df = pd.read_csv('./Counties/'+ loc +'.csv')
    tmp_df['date'] = pd.to_datetime(tmp_df['date']).dt.strftime('%Y-%m').astype(str)
    tmp_df = tmp_df.groupby(by='date').mean()

    tmp_df['County'] = loc
    tmp_df.reset_index(inplace=True)
    tmp_df = tmp_df.rename(columns = {'index':'date'})

    tmp_df = tmp_df[['date','temperature (degC)','wind_speed (m/s)','total_precipitation (mm of water equivalent)','relative_humidity (0-1)','surface_pressure (Pa)','humidex (degC)','County']]
    tmp_df.to_csv('Counties_Cleaned/' + loc + '.csv')

In [24]:
PerCounty_Data = []

for loc in Counties:
    tmp_df = pd.read_csv('Counties_Cleaned/' + loc + '.csv', index_col=None)
    tmp_df = tmp_df[['date','temperature (degC)','wind_speed (m/s)','total_precipitation (mm of water equivalent)','relative_humidity (0-1)','surface_pressure (Pa)','humidex (degC)','County']]
    PerCounty_Data.append(tmp_df)

weather_data = pd.concat(PerCounty_Data, ignore_index=True)

In [25]:
weather_data["data_id"] = weather_data["County"] + '-'+ weather_data["date"]
weather_data = weather_data.set_index('data_id')

In [15]:
# weather_data.to_csv('Weather_data.csv')

#### Combined Data

In [33]:
all_data = pd.concat([wildfire_data, weather_data], axis=1)
# all_data = all_data.fillna(0)
all_data['y'] = all_data['incident_acres_burned']>0

In [34]:
all_data = all_data.drop(columns=['calfire_incident','incident_acres_burned','incident_containment'])

In [35]:
all_data.sort_values(by=['date','County'])

,date,temperature (degC),wind_speed (m/s),total_precipitation (mm of water equivalent),relative_humidity (0-1),surface_pressure (Pa),humidex (degC),County,y
Alameda-2013-01,2013-01,7.889032,1.757097,0.028710,0.759032,101213.421613,6.898710,Alameda,False
Alpine-2013-01,2013-01,5.518710,2.853871,0.071935,0.560968,85811.907742,2.783226,Alpine,False
Amador-2013-01,2013-01,12.397419,4.447419,0.096774,0.821935,101546.030968,13.470968,Amador,False
Butte-2013-01,2013-01,27.599355,1.026452,0.231935,0.792903,100825.498710,38.547097,Butte,False
Calaveras-2013-01,2013-01,6.312258,1.019032,0.038387,0.691290,96573.848387,4.515806,Calaveras,False
...,...,...,...,...,...,...,...,...,...
Riverside-2021-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
San Diego-2009-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
Santa Cruz-2021-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
State of Oregon-2019-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [31]:
all_data.y.value_counts()

False    4598
True      982
Name: y, dtype: int64

## Fire Probability
- features: temperature, wind speed, total precipitation, relative_humidity
- output: probability of wildfire happenings

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [37]:
model_data = all_data.dropna()

In [46]:
# select features and prediction
model_X = model_data[['temperature (degC)', 'wind_speed (m/s)', 'total_precipitation (mm of water equivalent)', 'relative_humidity (0-1)']]
model_y = model_data[['y']].astype(int)

In [47]:
# split test and train
X_train, X_test, y_train, y_test = train_test_split(model_X, model_y, test_size=0.2, shuffle=True)

In [76]:
# build model
LR = LogisticRegression(max_iter=1000).fit(X_train, y_train.to_numpy().reshape(-1))

In [77]:
# test prediction
y_pred = LR.predict(X_test)
sum((y_pred- y_test.to_numpy().reshape(1097))==0)/len(y_pred)

0.8222424794895169

In [125]:
def predict_prob(model, temp, windSpeed, precip, humid):
    '''
    given a model, [temp, windspeed, precipitation, humidity], predict the probability of wildfire
    @temp: temperature
    @windspeed: windspeed
    @precip: precipitation
    @humid: humidity
    '''
    assert precip >= 0
    assert 0 <= humid <= 1
    feature = np.array([temp, windSpeed, precip, humid]).reshape(1, -1)
    prob = model.predict_proba(feature)[0][0]
    print(f'probability of wildfire: {1-prob}')
    return 1-prob

In [126]:
def predict_prob_interact(temp, windSpeed, precip, humid):
    '''
    given a model, [temp, windspeed, precipitation, humidity], predict the probability of wildfire
    @temp: temperature
    @windspeed: windspeed
    @precip: precipitation
    @humid: humidity
    '''
    assert precip >= 0
    assert 0 <= humid <= 1
    model=LR
    feature = np.array([temp, windSpeed, precip, humid]).reshape(1, -1)
    prob = model.predict_proba(feature)[0][0]
    print(f'probability of wildfire: {1-prob}')
    return 1-prob

In [127]:
predict_prob(LR, temp=7.889032, windSpeed=1.757097, precip=0.1, humid=0.3)

probability of wildfire: 0.039865195159208056


0.039865195159208056

In [34]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [129]:
interact(predict_prob_interact, temp=(0,40.,1), windSpeed=(0,10,0.1), precip=(0,1,0.1), humid=(0,1,0.1));

interactive(children=(FloatSlider(value=20.0, description='temp', max=40.0, step=1.0), FloatSlider(value=5.0, …

## Expected number of fire per county

In [3]:
my_data = pd.read_csv('./Processed_County.csv')

In [5]:
my_data.head()

,Unnamed: 0,incident_latitude,incident_longitude,incident_dateonly_created,incident_control,incident_containment,incident_acres_burned,incident_county,incident_name,calfire_incident,is_active,incident_id,incident_type,County
0,0,40.774000,-122.309000,2017-10-31,NaN,100.0,37.0,Shasta,Bridge Fire,False,N,2ca11d45-8139-4c16-8af0-880d99b21e82,NaN,Shasta
1,1,1.000000,1.000000,2009-05-24,NaN,100.0,122.0,San Diego,Pala Fire,True,N,8f61f461-552d-4538-b186-35ab030da416,Wildfire,San Diego
2,2,36.602575,-118.016510,2013-02-24,NaN,100.0,406.0,Inyo,River Fire,True,N,094719ba-a47b-4abb-9ec5-a506b2b9fd23,NaN,Inyo
3,3,34.288877,-116.941311,2013-04-20,NaN,100.0,30.0,San Bernardino,Fawnskin Fire,False,N,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,NaN,San Bernardino
4,4,37.116295,-119.635004,2013-04-30,NaN,100.0,274.0,Madera,Gold Fire,True,N,357ffc13-bef9-48eb-810f-c5de851972eb,NaN,Madera


In [16]:
my_data['month']=pd.to_datetime(my_data['incident_dateonly_created']).dt.month
my_data['year']=pd.to_datetime(my_data['incident_dateonly_created']).dt.year

In [17]:
county_occurances = my_data[['month','year', 'County']]


In [23]:
total_res = county_occurances.groupby(county_occurances.columns.tolist(),as_index=False).size()

In [25]:
total_res.sort_values(by='size')

,month,year,County,size
0,1,2014,Fresno,1
616,8,2017,San Bernardino,1
619,8,2017,Santa Clara,1
622,8,2017,Sonoma,1
626,8,2017,Yuba,1
...,...,...,...,...
615,8,2017,Riverside,8
820,9,2019,Riverside,8
426,7,2017,San Bernardino,8
423,7,2017,Riverside,9


In [37]:
def pred_occurances(county, month):
    try:
        res = total_res.groupby(by=['County', 'month']).mean()['size']
        return res[county][month]
    except:
        return 0

In [38]:
Counties = ['Alameda','Alpine','Amador','Butte','Calaveras','Colusa','Contra Costa','Del Norte','El Dorado','Fresno','Glenn','Humboldt','Imperial','Inyo','Kern','Kings','Lake','Lassen','Los Angeles','Madera','Marin','Mariposa','Mendocino','Merced','Modoc','Mono','Monterey','Napa','Nevada','Orange','Placer','Plumas','Riverside','Sacramento','San Benito','San Bernardino','San Diego','San Francisco','San Joaquin','San Luis Obispo','San Mateo','Santa Barbara','Santa Clara','Santa Cruz','Shasta','Sierra','Siskiyou','Solano','Sonoma','Stanislaus','Sutter','Tehama','Trinity','Tulare','Tuolumne','Ventura','Yolo','Yuba']
interact(pred_occurances, county=Counties, month=(1,12,1))

interactive(children=(Dropdown(description='county', options=('Alameda', 'Alpine', 'Amador', 'Butte', 'Calaver…

<function __main__.pred_occurances(county, month)>

## Previous

In [63]:
X = all_data[['date','temperature (degC)','wind_speed (m/s)','total_precipitation (mm of water equivalent)','relative_humidity (0-1)','surface_pressure (Pa)','humidex (degC)','County']]
y = all_data.y

In [67]:
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X, y)
LR.predict(X.iloc[460:,:])
round(LR.score(X,y), 4)

ValueError: could not convert string to float: '2013-07'